<a href="https://colab.research.google.com/github/Riyan081/Python-CDAC/blob/main/practical_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/playground-series-s6e2/sample_submission.csv
/kaggle/input/playground-series-s6e2/train.csv
/kaggle/input/playground-series-s6e2/test.csv


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [ ]:
train_df = pd.read_csv("/kaggle/input/playground-series-s6e2/train.csv")
test_df  = pd.read_csv("/kaggle/input/playground-series-s6e2/test.csv")

test_df.head()

train_df.head()

,id,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium,Heart Disease
0,0,58,1,4,152,239,0,0,158,1,3.6,2,2,7,Presence
1,1,52,1,1,125,325,0,2,171,0,0.0,1,0,3,Absence
2,2,56,0,2,160,188,0,2,151,0,0.0,1,0,3,Absence
3,3,44,0,3,134,229,0,2,150,0,1.0,2,0,3,Absence
4,4,58,1,4,140,234,0,2,125,1,3.8,2,3,3,Presence


In [ ]:
TARGET = "Heart Disease"
ID_COL = "id"

X = train_df.drop([TARGET, ID_COL], axis=1)
y = train_df[TARGET]

X_test = test_df.drop(ID_COL, axis=1)

In [ ]:
le = LabelEncoder()
y = le.fit_transform(y)

y = pd.Series(y, name=TARGET)

dict(zip(le.classes_, le.transform(le.classes_)))

{'Absence': np.int64(0), 'Presence': np.int64(1)}

In [ ]:
cat_cols = [
    "Age",
    "Sex",
    "Chest pain type",
    "BP",
    "Cholesterol",
    "FBS over 120",
    "EKG results",
    "Max HR",
    "Exercise angina",
    "Slope of ST",
    "Number of vessels fluro",
    "Thallium"
]

cat_idx = [X.columns.get_loc(col) for col in cat_cols]

In [ ]:
xgb = XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric="logloss",
    random_state=42
)

lgb = LGBMClassifier(
    n_estimators=300,
    num_leaves=31,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

cat = CatBoostClassifier(
    iterations=300,
    depth=6,
    learning_rate=0.05,
    cat_features=cat_idx,
    loss_function="Logloss",
    verbose=0,
    random_state=42
)

models = {
    "XGBoost": xgb,
    "LightGBM": lgb,
    "CatBoost": cat
}


In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

cv_results = {}

for name, model in models.items():
    fold_losses = []

    for tr_idx, val_idx in skf.split(X, y):
        X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]

        model.fit(X_tr, y_tr)
        val_prob = model.predict_proba(X_val)[:, 1]
        fold_losses.append(log_loss(y_val, val_prob))

    cv_results[name] = np.mean(fold_losses)

cv_results_df = pd.DataFrame.from_dict(
    cv_results, orient="index", columns=["CV Log Loss"]
)

cv_results_df


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 225963, number of negative: 278037
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.056804 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 422
[LightGBM] [Info] Number of data points in the train set: 504000, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.448339 -> initscore=-0.207383
[LightGBM] [Info] Start training from score -0.207383
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 225963, number of negative: 278037
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.058911 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 417
[LightGBM] [Info] Number of data points in the train set: 504000, number of 

,CV Log Loss
XGBoost,0.268800
LightGBM,0.268977
CatBoost,0.267306


In [ ]:
for model in models.values():
    model.fit(X, y)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 282454, number of negative: 347546
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.065632 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 417
[LightGBM] [Info] Number of data points in the train set: 630000, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.448340 -> initscore=-0.207381
[LightGBM] [Info] Start training from score -0.207381


In [ ]:
test_probs = {}

for name, model in models.items():
    test_probs[name] = model.predict_proba(X_test)[:, 1]

In [ ]:
ensemble_avg_prob = np.mean(
    np.column_stack(list(test_probs.values())),
    axis=1
)

In [ ]:
weights = [0.3, 0.3, 0.4]

ensemble_weighted_prob = (
    weights[0]*test_probs["XGBoost"] +
    weights[1]*test_probs["LightGBM"] +
    weights[2]*test_probs["CatBoost"]
)


In [ ]:
blend_train = np.column_stack([
    models["XGBoost"].predict_proba(X)[:,1],
    models["LightGBM"].predict_proba(X)[:,1],
    models["CatBoost"].predict_proba(X)[:,1]
])

blend_test = np.column_stack([
    test_probs["XGBoost"],
    test_probs["LightGBM"],
    test_probs["CatBoost"]
])


In [ ]:
meta = LogisticRegression()
meta.fit(blend_train, y)

blend_prob = meta.predict_proba(blend_test)[:,1]


In [ ]:
# final_prob = blend_prob
# final_prob = ensemble_avg_prob
final_prob = ensemble_weighted_prob

final_pred = (final_prob > 0.5).astype(int)

In [ ]:
submission = pd.DataFrame({
    "id": test_df["id"],
    "Heart Disease": final_prob
})

submission.to_csv("submission.csv", index=False)
submission.head()

,id,Heart Disease
0,630000,0.947449
1,630001,0.008360
2,630002,0.987214
3,630003,0.005222
4,630004,0.197909
